### Pull Data from Reddit subreddit NYCapartments using Reddit API 

In [2]:
# imports
import json
import pandas as pd
import praw
import time

In [ ]:
# open credential file
with open('client_secrets.json', 'r') as f:
    app_creds = json.load(f)


In [8]:
# get app credentials
client_id = app_creds['client_id']
client_secret = app_creds['client_secret']
user_agent = app_creds['user_agent']
redirect_uri = app_creds['redirect_uri']
client_password = app_creds['password']
user_name = app_creds['user_name']

### Connect to Reddit API

In [9]:
# connect to Reddit API using PRAW
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    password=client_password,
    user_agent=user_agent,
    username=user_name,
)

print(reddit.read_only)

False


In [10]:
# add in astoriaapartments later
subreddit_topics = ['astoriaapartments', 'NYCapartments']

In [84]:
# Pull posts from last year NYCapartments subreddit
start_time = time.time()
post_list = []
for post in reddit.subreddit("NYCapartments").top(time_filter="year"):
# for post in reddit.subreddit("astoriaapartments+NYCapartments").top(time_filter="year"):
    post_dict = {}
    post_comments = []
    post_dict['subreddit'] = post.subreddit_name_prefixed
    post_dict['title'] = post.title
    post_dict['score'] = post.score
    post_dict['num_comments'] = post.num_comments
    post_dict['url'] = post.shortlink
    post.comments.replace_more(limit=None)
    for top_level_comment in post.comments:
        post_comments.append(top_level_comment.body)
    post_dict['comments'] = post_comments
    post_list.append(post_dict)

end_time = time.time()
total_time = round(((start_time - end_time)/60.0),2)
print(f"Finished pull data in {total_time} mins")

Finished pull data in -2.12 mins


In [85]:
len(post_list)

100

In [88]:
post_dict

{'subreddit': 'r/NYCapartments',
 'title': 'List of landlords you SHOULD rent from',
 'score': 178,
 'num_comments': 45,
 'url': 'https://redd.it/1fjyynz',
 'comments': ['Ray simone - Williamsburg, very humble guy. Has refused to raise rents on his tenants even though he could',
  'The best landlord you can hope for is one that actually follows the laws and regulations as they are written. They are actually pretty few and far between',
  'Eberhart brothers has been very good for such a large company I’ll say. My second year with them now',
  'Most of the landlords you’re seeking are small , three family owners like mine. I won’t tell you where I rent, and I’m assuming the other owners won’t. I’ll live here til I die',
  'Kibel and Rose Properties are both totally fine.\nI was able to get apartments with both directly from the landlord and avoid broker fees. \nThey are both corporately run and have dealt with any issues swiftly, I never had any headaches with either.',
  'Can be hit or 

In [89]:
# create dataframe
df = pd.DataFrame(post_list)

In [90]:
df.head()

,subreddit,title,score,num_comments,url,comments
0,r/NYCapartments,Midtown manhattan Studio with great view,1623,289,https://redd.it/1ditjh8,"[$5000 for a studio 😕, It comes with personal ..."
1,r/NYCapartments,"Spacious studio on the UWS, great location ste...",1432,57,https://redd.it/1at9s69,"[When can I move in?, Wow! What a steal!, Pets..."
2,r/NYCapartments,Those with high rise skyline apartments do you...,1337,249,https://redd.it/1fwnevs,[I didn’t live in one but I worked at an offic...
3,r/NYCapartments,The rent crisis in NYC is becoming increasingl...,1260,371,https://redd.it/1aq7si7,"[Two incomes no kids., Tell me about it, I lit..."
4,r/NYCapartments,Happy Monday everyone,942,199,https://redd.it/1auv5ws,[I think average salary context is needed. Tak...


In [92]:
df[df.num_comments==0].shape

(0, 6)

In [93]:
# write to csv
df.to_csv('nyc_apt.csv')

In [95]:
# write to text file
with open('reddit_posts_nyc_apt.txt', 'w') as out_file:
    for item in post_list:
        out_file.write(f"{item}\n")
